In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer
import networkx as nx
from textblob import TextBlob
from networkx.algorithms import bipartite
import nltk
from nltk import sent_tokenize, wordnet, word_tokenize
from nltk.corpus import wordnet as wn
from nltk.corpus import conll2000, stopwords
from pandas import HDFStore, DataFrame

OPEN THE ORIGINAL CSV

In [2]:
#read the comment file. maybe change the encoding..
comments=pd.read_csv('commenti.csv', encoding = "ISO-8859-1")
comments=comments.drop('Unnamed: 0', axis=1)
comments=comments.drop_duplicates()

C:\Users\giuli\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#drop parent_id, moderation status and drop NAs
comments=comments.drop(['parent_id', 'moderationStatus', 
                        'user_name'], axis=1)
comments=comments.dropna()

CREATE THE WHOLE GRAPH

In [4]:
#create a digraph
B = nx.DiGraph()
#add nodes of bipartite graph from users and videos
B.add_nodes_from(comments['user_channel_id'], bipartite=0)
B.add_nodes_from(comments['video_id'], bipartite=1)
#add edges
B.add_edges_from([(row['user_channel_id'], row['video_id']) for idx, row in comments.iterrows()])
#set position for the plot
pos = {node:[0, i] for i,node in enumerate(comments['user_channel_id'])}
pos.update({node:[1, i] for i,node in enumerate(comments['video_id'])})

#save outdegree of the nodes
degs=B.out_degree()

CREATE THE SUBGRAPH

In [5]:
#select the nodes we want to study
keep=[x for x in B.nodes() if degs[x]>146]

In [6]:
#create a new df where the user channels appears in the one with the conditions stated in 'keep'
queryDF=comments.loc[comments.user_channel_id.isin(keep)]

B1 = nx.DiGraph()
#add nodes of bipartite graph from users and videos
B1.add_nodes_from(queryDF['user_channel_id'], bipartite=0)
B1.add_nodes_from(queryDF['video_id'], bipartite=1)
#add edges
B1.add_edges_from([(row['user_channel_id'], row['video_id']) for idx, row in queryDF.iterrows()])
#set position for the plot
pos = {node:[0, i] for i,node in enumerate(queryDF['user_channel_id'])}
pos.update({node:[1, i] for i,node in enumerate(queryDF['video_id'])})

CREATE CORPUS

In [ ]:
#create corpus as list of comment of every user from the net 
corpus=[]
for user in users:
    his_comm=comments.loc[comments.user_channel_id==user]
    his_list=[]
    for i in his_comm.index: 
        temp=(his_comm.comment_text_original[i])
        his_list.append(temp)
    corpus.append(''.join(his_list))
    
#save it 
"""corpusSave=pd.DataFrame(corpus)
corpusSave.to_pickle('corpus.pkl')"""

CLEAN THE CORPUS

In [ ]:
#clean the corpus
clean=[]
#for each user in the corpus
for user in range(len(corpus)):
    his_clean=[]
    #tokenize the text of this user and create a list of words
    tok=nltk.RegexpTokenizer(r'\w+').tokenize(text=corpus[user])
    for w in range(len(tok)):
        #for each word, make it lower
        tok[w]=tok[w].lower()
        #check if stopword
        if tok[w] not in stopwords.words('english'):
            his_clean.append(WordNetLemmatizer().lemmatize(tok[w]))
    clean.append(his_clean)

In [ ]:
#join all cleaned text together to get to a cleaned corpus
cleaned_corpus=[]
for i in clean:
    cleaned_corpus.append(' '.join(i))

In [ ]:
#IDF AND IF SCORE, NOT USED ANYMORE.
#IDF score
IDF_dict={}
for i in set([item for sublist in clean for item in sublist]):
    IDF_dict[i]=IDF(i,clean)
    
#TF score
scores={}
for i in set(clean):
    scores[i]=TF(i,clean)

SAVE RESULTS

In [7]:
#save networks
nx.write_adjlist(B1, 'B1.txt')
nx.write_adjlist(B, 'full_net.txt')

In [8]:
#save compressed version 
nx.write_adjlist(B1, 'B1.gz')
nx.write_adjlist(B, 'full_net.gz')

In [9]:
#save comments dataframe
comments.to_pickle('Comments_df.pkl')

In [ ]:
clean_corp_save=pd.DataFrame(cleaned_corpus)
clean_corp_save.to_pickle('cleaned_corpus.pkl')

#save the scores, no more needed
cleanSave=pd.DataFrame(clean)
cleanSave.to_pickle('clean.pkl')


scoreSave=pd.DataFrame.from_dict(scores, orient='index', columns=['count'])
scoreSave.to_pickle('scores.pkl')

scoreSave=pd.DataFrame.from_dict(IDF_dict, orient='index', columns=['count'])
scoreSave.to_pickle('IDF_dict.pkl')